# Compute the Histogram of Gradients (HOG)

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def show_rgb(img):
    # Change the order of the channels
    rgbimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(rgbimg)
    plt.show()
    
# read image
img = cv2.imread('images/owl.png')
# show image
show_rgb(img)

In [ ]:
from numpy.linalg import norm
def hog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bin_n = 9 # Number of bins
    bin = np.int32(bin_n*ang/(2*np.pi))
    
    bin_cells = []
    mag_cells = []

    cellx = celly = 8

    for i in range(0,img.shape[0]/celly):
        for j in range(0,img.shape[1]/cellx):
            bin_cells.append(bin[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])
            mag_cells.append(mag[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])   

    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)

    # transform to Hellinger kernel
    eps = 1e-7
    hist /= hist.sum() + eps
    hist = np.sqrt(hist)
    hist /= norm(hist) + eps

    return hist
    
hist = hog(img)
print(hist.shape)
print(hist)

In [ ]:
fig = plt.figure(figsize=(10,10))
from skimage import color, feature, exposure
gray = color.rgb2gray(img)
_, imghog = feature.hog(gray, orientations=9, 
                        pixels_per_cell=(30, 30), #4, 4 
                        cells_per_block=(2, 2), #2, 2
                        visualise=True)
resc_hog = exposure.rescale_intensity(imghog, in_range=(0, 0.08))
plt.imshow(resc_hog, cmap='gray')
plt.show()